In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set Google API key (replace with your actual key or use .env file)
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", "your_google_api_key_here")

## Install libraries

In [4]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [5]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [2]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 12.9 MB/s eta 0:00:00


## Step 1a - Indexing (Document Ingestion)

In [ ]:
import time
import random
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.proxies import WebshareProxyConfig
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

# Proxy config
proxy_username = "yourapikey"
proxy_password = "yourapisecret"

video_id = "wjZofJX0v4M"

ytt_api = YouTubeTranscriptApi(
    proxy_config=WebshareProxyConfig(
        proxy_username=proxy_username,
        proxy_password=proxy_password,
    )
)

def fetch_transcript_with_retry(video_id, max_retries=5):
    for attempt in range(max_retries):
        try:
            transcript_list = ytt_api.fetch(video_id, languages=['en'])
            return " ".join(chunk.text for chunk in transcript_list)
        except (TranscriptsDisabled, NoTranscriptFound) as e:
            raise e  # These are permanent errors; don’t retry
        except Exception as e:
            if "quota" in str(e).lower() or "429" in str(e) or "temporarily" in str(e).lower():
                wait = 2 ** attempt + random.uniform(0, 0.5)
                print(f"⏳ Retry {attempt+1}/{max_retries} after {wait:.2f}s due to error: {e}")
                time.sleep(wait)
            else:
                raise e  # Unknown error, re-raise

try:
    transcript = fetch_transcript_with_retry(video_id)
    print("✅ Transcript fetched successfully!")
    print(transcript)

except TranscriptsDisabled:
    print("🚫 No captions available for this video.")
except NoTranscriptFound:
    print("❗ Transcript not found in the specified language.")
except Exception as e:
    print(f"❌ Failed to fetch transcript: {e}")


✅ Transcript fetched successfully!
The initials GPT stand for Generative Pretrained Transformer. So that first word is straightforward enough, these are bots that generate new text. Pretrained refers to how the model went through a process of learning from a massive amount of data, and the prefix insinuates that there's more room to fine-tune it on specific tasks with additional training. But the last word, that's the real key piece. A transformer is a specific kind of neural network, a machine learning model, and it's the core invention underlying the current boom in AI. What I want to do with this video and the following chapters is go through a visually-driven explanation for what actually happens inside a transformer. We're going to follow the data that flows through it and go step by step. There are many different kinds of models that you can build using transformers. Some models take in audio and produce a transcript. This sentence comes from a model going the other way around, p

In [ ]:
import time
import random
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.proxies import WebshareProxyConfig
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

# Proxy config
proxy_username = "yourapikey"
proxy_password = "yourapisecret"

video_id = "J5_-l7WIO_w"

ytt_api = YouTubeTranscriptApi(
    proxy_config=WebshareProxyConfig(
        proxy_username=proxy_username,
        proxy_password=proxy_password,
    )
)

def list_transcripts_with_retry(video_id, max_retries=5):
    for attempt in range(max_retries):
        try:
            return ytt_api.list(video_id)
        except (TranscriptsDisabled, NoTranscriptFound) as e:
            raise e  # No use retrying these
        except Exception as e:
            if "quota" in str(e).lower() or "429" in str(e) or "temporarily" in str(e).lower():
                wait = 2 ** attempt + random.uniform(0, 0.5)
                print(f"⏳ Retry {attempt+1}/{max_retries} in {wait:.2f}s due to error: {e}")
                time.sleep(wait)
            else:
                raise e  # Unknown error — let it crash

try:
    transcript_list = list_transcripts_with_retry(video_id)

    print("📋 Available transcripts for this video:")
    for transcript in transcript_list:
        lang = transcript.language
        lang_code = transcript.language_code
        is_generated = " (auto-generated)" if transcript.is_generated else ""
        print(f"  - {lang} [{lang_code}]{is_generated}")

except TranscriptsDisabled:
    print("🚫 No captions available for this video.")
except NoTranscriptFound:
    print("❗ No transcript found in any language.")
except Exception as e:
    print(f"❌ Failed to list transcripts: {e}")


⏳ Retry 1/5 in 1.48s due to error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /sorry/index?continue=https://www.youtube.com/watch%3Fv%3DJ5_-l7WIO_w&q=EgQXX5aRGOXX_cMGIjB7Zie-haH4XwfHm1iegk5-aqvkBoMTa4ZZywbOycW61w94cu4FQk8sURvHOXxTSLEyAnJSWgFD (Caused by ResponseError('too many 429 error responses'))
📋 Available transcripts for this video:
  - Hindi (auto-generated) [hi] (auto-generated)


In [90]:
import time
import random

def retry_with_backoff(func, *args, max_retries=5, handle_errors=None, **kwargs):
    """
    Retry a function with exponential backoff and jitter.

    Parameters:
        func (callable): The function to retry.
        *args: Positional arguments for the function.
        max_retries (int): Maximum retry attempts.
        handle_errors (tuple): Error types or string patterns to retry on.
        **kwargs: Keyword arguments for the function.

    Returns:
        The return value of `func` if successful.

    Raises:
        The last exception if all retries fail.
    """
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if handle_errors:
                # If error match is a string or tuple of strings
                if isinstance(handle_errors, tuple):
                    should_retry = any(pattern.lower() in str(e).lower() for pattern in handle_errors)
                else:
                    should_retry = handle_errors.lower() in str(e).lower()
            else:
                should_retry = True  # retry all by default

            if should_retry:
                wait = 2 ** attempt + random.uniform(0, 0.5)
                print(f"⏳ Retry {attempt+1}/{max_retries} after {wait:.2f}s due to: {e}")
                time.sleep(wait)
            else:
                raise e  # not a retriable error
    raise Exception(f"❌ All {max_retries} retries failed.")


In [53]:
transcript_list[0]

FetchedTranscriptSnippet(text='the following is a conversation with', start=0.08, duration=3.44)

In [54]:
# Save transcript to file
with open("transcript.txt", "w", encoding="utf-8") as f:
    f.write(transcript)
print("Transcript saved to transcript.txt")

Transcript saved to transcript.txt


In [55]:
transcript

'The initials GPT stand for Generative Pretrained Transformer. So that first word is straightforward enough, these are bots that generate new text. Pretrained refers to how the model went through a process of learning from a massive amount of data, and the prefix insinuates that there\'s more room to fine-tune it on specific tasks with additional training. But the last word, that\'s the real key piece. A transformer is a specific kind of neural network, a machine learning model, and it\'s the core invention underlying the current boom in AI. What I want to do with this video and the following chapters is go through a visually-driven explanation for what actually happens inside a transformer. We\'re going to follow the data that flows through it and go step by step. There are many different kinds of models that you can build using transformers. Some models take in audio and produce a transcript. This sentence comes from a model going the other way around, producing synthetic speech just

## Step 1b - Indexing (Text Splitting)

In [56]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [57]:
len(chunks)

37

In [59]:
chunks[12]

Document(metadata={}, page_content="line is described by two continuous parameters, say the slope and the y-intercept, and the goal of linear regression is to determine those parameters to closely match the data. Needless to say, deep learning models get much more complicated. GPT-3, for example, has not two, but 175 billion parameters. But here's the thing, it's not a given that you can create some giant model with a huge number of parameters without it either grossly overfitting the training data or being completely intractable to train. Deep learning describes a class of models that in the last couple decades have proven to scale remarkably well. What unifies them is that they all use the same training algorithm, it's called backpropagation, we talked about it in previous chapters, and the context that I want you to have as we go in is that in order for this training algorithm to work well at scale, these models have to follow a certain specific format. And if you know this format g

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [94]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [96]:
vec

[-0.0026646258775144815,
 -0.008441368117928505,
 0.02872495912015438,
 -0.09430649876594543,
 -0.018615247681736946,
 0.012315296567976475,
 -0.005853960756212473,
 -0.00453009270131588,
 -0.009130255319178104,
 0.021546021103858948,
 0.0036647983361035585,
 -0.0011667441576719284,
 0.01610543578863144,
 0.0013404052006080747,
 0.12616394460201263,
 -0.002917769132182002,
 0.006669206544756889,
 -0.0002157641138182953,
 -0.0010517803020775318,
 0.011316517367959023,
 0.00012958302977494895,
 0.01074906438589096,
 -0.00087612250354141,
 -0.019158467650413513,
 0.007817811332643032,
 -0.01879890449345112,
 0.034895747900009155,
 0.006449114065617323,
 0.052493829280138016,
 0.0009636892937123775,
 -0.015054444782435894,
 0.01169053465127945,
 0.005224000662565231,
 0.008823500014841557,
 0.0229586623609066,
 0.008717045187950134,
 -0.007572086527943611,
 0.007365094963461161,
 -0.015274212695658207,
 0.006710423622280359,
 0.018370630219578743,
 -0.002816644264385104,
 0.010763252153992

In [62]:
import time
import random

valid_texts = []
valid_embeddings = []

def embed_with_retry(text, max_retries=5):
    for attempt in range(max_retries):
        try:
            return embedding_model.embed_query(text)
        except Exception as e:
            if "quota" in str(e).lower() or "429" in str(e):
                wait = 2 ** attempt + random.uniform(0, 0.5)
                print(f"⏳ Quota error, retrying in {wait:.2f}s (attempt {attempt+1})")
                time.sleep(wait)
            else:
                raise e  # not a quota error

for idx, doc in enumerate(chunks):
    try:
        text = doc.page_content
        vec = embed_with_retry(text)
        valid_texts.append(text)
        valid_embeddings.append(vec)
        print(f"✅ Embedded {idx+1}/{len(chunks)}")
    except Exception as e:
        print(f"❌ Skipped doc {idx+1}: {e}")
        continue


✅ Embedded 1/37
✅ Embedded 2/37
✅ Embedded 3/37
✅ Embedded 4/37
✅ Embedded 5/37
✅ Embedded 6/37
✅ Embedded 7/37
✅ Embedded 8/37
✅ Embedded 9/37
✅ Embedded 10/37
✅ Embedded 11/37
✅ Embedded 12/37
✅ Embedded 13/37
✅ Embedded 14/37
✅ Embedded 15/37
✅ Embedded 16/37
✅ Embedded 17/37
✅ Embedded 18/37
✅ Embedded 19/37
✅ Embedded 20/37
✅ Embedded 21/37
✅ Embedded 22/37
✅ Embedded 23/37
✅ Embedded 24/37
✅ Embedded 25/37
✅ Embedded 26/37
✅ Embedded 27/37
✅ Embedded 28/37
✅ Embedded 29/37
✅ Embedded 30/37
✅ Embedded 31/37
✅ Embedded 32/37
✅ Embedded 33/37
✅ Embedded 34/37
✅ Embedded 35/37
✅ Embedded 36/37
✅ Embedded 37/37


In [63]:
valid_embeddings

[[-0.005865778774023056,
  0.025624610483646393,
  0.01708146370947361,
  -0.07048432528972626,
  -0.010786155238747597,
  0.01630290225148201,
  0.006251067854464054,
  0.0016525144455954432,
  0.014591560699045658,
  0.02977689541876316,
  0.0004744240141008049,
  0.007242704275995493,
  0.001421187655068934,
  0.013585450127720833,
  0.11945798993110657,
  0.003000192577019334,
  0.017825154587626457,
  -0.005730748642235994,
  -0.00022407261712942272,
  -0.017405178397893906,
  0.013864398002624512,
  0.003491042647510767,
  -0.011506914161145687,
  0.01137828454375267,
  -0.012296777218580246,
  0.0004221741983201355,
  0.02801896631717682,
  0.011555423960089684,
  0.058290205895900726,
  -0.009623079560697079,
  -0.008198481984436512,
  0.0036296662874519825,
  -0.004201022442430258,
  0.01075056940317154,
  -0.004463236778974533,
  0.012180632911622524,
  0.006895562168210745,
  -0.0036032721400260925,
  -0.02037718892097473,
  0.01614750362932682,
  -0.0030760967638343573,
  -

In [65]:
# Extract texts and metadata from chunks
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks] # If your chunks have metadata
print(type(texts))
# Create pairs of (text, embedding)
text_embedding_pairs = list(zip(texts, valid_embeddings))

# Initialize the embedding model for future queries
# (Use the same model you used to generate valid_embeddings)
embedding_model_for_queries = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Replace the original line with this:
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    embedding=embedding_model_for_queries,
    metadatas=metadatas # Optional: include if your chunks had metadata
)

print("FAISS vector store successfully created using pre-computed embeddings!")

print("✅ FAISS vector store created successfully with", len(valid_texts), "documents.")

<class 'list'>
FAISS vector store successfully created using pre-computed embeddings!
✅ FAISS vector store created successfully with 37 documents.


In [66]:
texts[0]

"The initials GPT stand for Generative Pretrained Transformer. So that first word is straightforward enough, these are bots that generate new text. Pretrained refers to how the model went through a process of learning from a massive amount of data, and the prefix insinuates that there's more room to fine-tune it on specific tasks with additional training. But the last word, that's the real key piece. A transformer is a specific kind of neural network, a machine learning model, and it's the core invention underlying the current boom in AI. What I want to do with this video and the following chapters is go through a visually-driven explanation for what actually happens inside a transformer. We're going to follow the data that flows through it and go step by step. There are many different kinds of models that you can build using transformers. Some models take in audio and produce a transcript. This sentence comes from a model going the other way around, producing synthetic speech just fro

In [67]:
vector_store.index_to_docstore_id

{0: 'ec0d80f4-13dc-41a9-9319-ee625b2efb02',
 1: 'e04dc10b-357f-4462-b1ee-50defca333f8',
 2: '86484b50-cccd-4150-b1c7-a9ce2267e21c',
 3: '7bda5bf3-2808-479e-8bb8-1b532ab5bf17',
 4: '75d05408-a241-4783-9235-39d486edcfd1',
 5: '8148428b-7f5a-460a-860b-24ea69489ee8',
 6: '8e0cf266-53d8-4b74-a188-d14660b98dd2',
 7: 'bafbf737-d6c2-4061-9cbd-d3e5f7670bc7',
 8: 'e08995d6-5ce2-4eb2-b4e0-249b5e031b2b',
 9: '592f309e-38eb-4c69-8a72-c27a6a17e345',
 10: '7fa220cf-3865-408c-8c7e-118a6637ccc1',
 11: 'e22bcc34-e60e-47cd-a6f1-e9c8683bf3f2',
 12: '21360482-bdce-488d-a612-499c636825e7',
 13: '62bf1347-225e-473c-b0ff-7a572f819b53',
 14: '5776a32a-d0ce-44c5-9245-8cb11842a971',
 15: 'fbb665f2-abe9-4958-93e6-c0863dd4b578',
 16: '8d13f4d0-723a-4243-ac00-9e9a2e026795',
 17: 'a33a873a-6f06-4106-b846-65bc17eed0b9',
 18: 'e8b6325a-45af-4103-872a-2b32f4858ded',
 19: 'ca91f6f4-9cf1-4602-8980-f4c92610c95e',
 20: '10596c5e-d37c-4377-a7b1-f1352b646c62',
 21: '0e63ec5c-2cb0-4c34-8b9d-3eedde25e72b',
 22: '72095c6f-e3e5-

## Step 2 - Retrieval

In [68]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [69]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x798abd0c0690>, search_kwargs={'k': 4})

In [70]:
retriever.invoke('What is deepmind')

[Document(id='e22bcc34-e60e-47cd-a6f1-e9c8683bf3f2', metadata={}, page_content='this for granted these days, but the idea with machine learning is that rather than trying to explicitly define a procedure for how to do that task in code, which is what people would have done in the earliest days of AI, instead you set up a very flexible structure with tunable parameters, like a bunch of knobs and dials, and then, somehow, you use many examples of what the output should look like for a given input to tweak and tune the values of those parameters to mimic this behavior. For example, maybe the simplest form of machine learning is linear regression, where your inputs and outputs are each single numbers, something like the square footage of a house and its price, and what you want is to find a line of best fit through this data, you know, to predict future house prices. That line is described by two continuous parameters, say the slope and the y-intercept, and the goal of linear regression is

## Step 3 - Augmentation

In [71]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

In [72]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [73]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [74]:
retrieved_docs

[Document(id='af60ab29-83ad-40c6-b6a0-3825d0d10dcf', metadata={}, page_content="review by Patreon supporters. A final version should be up in public in a week or two, it usually depends on how much I end up changing based on that review. In the meantime, if you want to dive into attention, and if you want to help the channel out a little bit, it's there waiting."),
 Document(id='592f309e-38eb-4c69-8a72-c27a6a17e345', metadata={}, page_content="but at a high level this is the idea. In this chapter, you and I are going to expand on the details of what happens at the very beginning of the network, at the very end of the network, and I also want to spend a lot of time reviewing some important bits of background knowledge, things that would have been second nature to any machine learning engineer by the time transformers came around. If you're comfortable with that background knowledge and a little impatient, you could probably feel free to skip to the next chapter, which is going to focus 

In [75]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"review by Patreon supporters. A final version should be up in public in a week or two, it usually depends on how much I end up changing based on that review. In the meantime, if you want to dive into attention, and if you want to help the channel out a little bit, it's there waiting.\n\nbut at a high level this is the idea. In this chapter, you and I are going to expand on the details of what happens at the very beginning of the network, at the very end of the network, and I also want to spend a lot of time reviewing some important bits of background knowledge, things that would have been second nature to any machine learning engineer by the time transformers came around. If you're comfortable with that background knowledge and a little impatient, you could probably feel free to skip to the next chapter, which is going to focus on the attention blocks, generally considered the heart of the transformer. After that, I want to talk more about these multi-layer perceptron blocks, how trai

In [76]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [77]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      review by Patreon supporters. A final version should be up in public in a week or two, it usually depends on how much I end up changing based on that review. In the meantime, if you want to dive into attention, and if you want to help the channel out a little bit, it's there waiting.\n\nbut at a high level this is the idea. In this chapter, you and I are going to expand on the details of what happens at the very beginning of the network, at the very end of the network, and I also want to spend a lot of time reviewing some important bits of background knowledge, things that would have been second nature to any machine learning engineer by the time transformers came around. If you're comfortable with that background knowledge and a little impatient, you could probably feel free to skip to the next ch

## Step 4 - Generation

In [78]:
answer = llm.invoke(final_prompt)
print(answer.content)

I don't know.


## Building a Chain

In [79]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [80]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [81]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [82]:
parallel_chain.invoke('who is Demis')

{'context': "review by Patreon supporters. A final version should be up in public in a week or two, it usually depends on how much I end up changing based on that review. In the meantime, if you want to dive into attention, and if you want to help the channel out a little bit, it's there waiting.\n\nbigger than 2. There's no mathematical reason for this, it's just an arbitrary constraint imposed to keep their tool from being seen generating things that are too nonsensical. So if you're curious, the way this animation is actually working is I'm taking the 20 most probable next tokens that GPT-3 generates, which seems to be the maximum they'll give me, and then I tweak the probabilities based on an exponent of 1/5. As another bit of jargon, in the same way that you might call the components of the output of this function probabilities, people often refer to the inputs as logits, or some people say logits, some people say logits, I'm gonna say logits. So for instance, when you feed in som

In [83]:
parser = StrOutputParser()

In [84]:
main_chain = parallel_chain | prompt | llm | parser

In [86]:
main_chain.invoke('Is Transformers in the video? If yes, then give what is transformers as explained in the video.')

"Yes, transformers are in the video. A transformer is a specific kind of neural network, a machine learning model, and it's the core invention underlying the current boom in AI. There are many different kinds of models that you can build using transformers. Some models take in audio and produce a transcript. The original transformer introduced in 2017 by Google was invented for the specific use case of translating text from one language into another. The variant that the video will focus on, which is the type that underlies tools like ChatGPT, will be a model that's trained to take in a piece of text, maybe even with some surrounding images or sound accompanying it, and produce a prediction for what comes next in the passage. That prediction takes the form of a probability."